# Project 3

### Aileen Yang cy2830

In [1]:
import plotly.io as pio
pio.renderers.default = "notebook_connected+plotly_mimetype"

**Dataset(s) to be used:** <br>
1. Counts and rates of asthma emergency department visits by zip code, California
https://healthdata.gov/State/Asthma-Emergency-Department-Visit-Rates/28nb-65xq/about_data 

2. California FIRE Damage Inspection Data
https://hub-calfire-forestry.hub.arcgis.com/datasets/cal-fire-damage-inspection-dins-data/about <br>


**Analysis question:** Do California areas that experience more wildfire activity have higher rates of asthma emergency department visits in children in the same year? Do these associations differ between children and adults?

**Columns that will (likely) be used:**

From asthma dataset:
- Year
- Zip Code
- County
- Age_Group
- Number_of_Asthma_ED_Visits
- Age_Adjusted_Rate_of_Asthma_ED_V

From wildfire dataset:
- OBJECTID (unique structure record)
- Incident Name
- Incident Start Date
- County

**Columns to be used to merge/join them:**
  - Asthma: Year, County
  - Wildfire: Year (from Incident Start Date), County

**Hypothesis**: California counties with more wildfire activity in a given year will have higher age-adjusted rates of asthma ED visits, particularly among children. 

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats

**Import & Cleansing of Asthma Data**

In [3]:
asthma = pd.read_csv('asthmaedvisitrates-by-zipcode.csv')
asthma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19190 entries, 0 to 19189
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Year                              19190 non-null  int64  
 1   Zip_Code                          19190 non-null  int64  
 2   County                            19190 non-null  object 
 3   Age_Group                         19190 non-null  object 
 4   Number_of_Asthma_ED_Visits        19190 non-null  int64  
 5   Age_Adjusted_Rate_of_Asthma_ED_V  19190 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 899.7+ KB


In [4]:
# Rename Columns
asthma = asthma.rename(columns={'Zip_Code':'Zip Code'})
asthma = asthma.rename(columns={'Age_Adjusted_Rate_of_Asthma_ED_V': 'Age-adjusted Rate'})
asthma = asthma.rename(columns={'Number_of_Asthma_ED_Visits': 'total_ed_visits'})

In [5]:
asthma["County"] = asthma["County"].astype(str).str.strip()
asthma = asthma.dropna(subset=["Year", "County", "Age_Group", "Age-adjusted Rate"]).copy()

I then aggregated them by age group, (child and adult), which will be used later for comparison later.

In [6]:
# Aggregate ED visits for each county
asthma_county = (
    asthma
    .groupby(["Year", "County", "Age_Group"])
    .agg(
        total_ed_visits=("total_ed_visits", "sum")
    )
    .reset_index()
)

asthma_county.head()


,Year,County,Age_Group,total_ed_visits
0,2013,Alameda,Adult,6653
1,2013,Alameda,Child,3213
2,2013,Amador,Adult,195
3,2013,Amador,Child,12
4,2013,Butte,Adult,648


**Importing Wildfire Dataset**

In [7]:
wildfire = pd.read_csv('Cali_Fire.csv')
wildfire.head()

/var/folders/sg/bpgt4bnn1qbfgvlc96v1x2m00000gn/T/ipykernel_8050/1965711151.py:1: DtypeWarning:

Columns (12,36,37) have mixed types. Specify dtype option on import or set low_memory=False.



,OBJECTID,* Damage,* Street Number,* Street Name,"* Street Type (e.g. road, drive, lane, etc.)","Street Suffix (e.g. apt. 23, blding C)",* City,State,Zip Code,* CAL FIRE Unit,...,Fire Name (Secondary),APN (parcel),Assessed Improved Value (parcel),Year Built (parcel),Site Address (parcel),GLOBALID,Latitude,Longitude,x,y
0,1,No Damage,8376.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090290,510000.0,1997.0,8376 QUAIL CANYON RD VACAVILLE CA 95688,e1919a06-b4c6-476d-99e5-f0b45b070de8,38.474960,-122.044465,-1.358593e+07,4.646741e+06
1,2,Affected (1-9%),8402.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090270,573052.0,1980.0,8402 QUAIL CANYON RD VACAVILLE CA 95688,b090eeb6-5b18-421e-9723-af7c9144587c,38.477442,-122.043252,-1.358579e+07,4.647094e+06
2,3,No Damage,8430.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090310,350151.0,2004.0,8430 QUAIL CANYON RD VACAVILLE CA 95688,268da70b-753f-46aa-8fb1-327099337395,38.479358,-122.044585,-1.358594e+07,4.647366e+06
3,4,No Damage,3838.0,Putah Creek,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0103010240,134880.0,1981.0,3838 PUTAH CREEK RD WINTERS CA 95694,64d4a278-5ee9-414a-8bf4-247c5b5c60f9,38.487313,-122.015115,-1.358266e+07,4.648497e+06
4,5,No Damage,3830.0,Putah Creek,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0103010220,346648.0,1980.0,3830 PUTAH CREEK RD WINTERS CA 95694,1b44b214-01fd-4f06-b764-eb42a1ec93d7,38.485636,-122.016122,-1.358277e+07,4.648259e+06


In [8]:
wildfire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130722 entries, 0 to 130721
Data columns (total 46 columns):
 #   Column                                                        Non-Null Count   Dtype  
---  ------                                                        --------------   -----  
 0   OBJECTID                                                      130722 non-null  int64  
 1   * Damage                                                      130722 non-null  object 
 2   * Street Number                                               126302 non-null  float64
 3   * Street Name                                                 125236 non-null  object 
 4   * Street Type (e.g. road, drive, lane, etc.)                  116260 non-null  object 
 5   Street Suffix (e.g. apt. 23, blding C)                        62017 non-null   object 
 6   * City                                                        98991 non-null   object 
 7   State                                                   

There are 45 columns in total for this dataset. For clarity, only essential columns that are useful for analysis will be kept. A new column 'Year' will be derived from the incident date to match the Asthma dataset year.

In [9]:
# Drop noise columns
wildfire = wildfire.rename(columns = {'* Incident Name':'Incident Name'})
wildfire = wildfire[["OBJECTID", "Incident Name", "Incident Start Date", "County"]].copy()

# Extract year to match the format of asthma years
wildfire["Incident Start Date"] = pd.to_datetime(
    wildfire["Incident Start Date"], errors="coerce"
)
wildfire = wildfire.dropna(subset=["Incident Start Date"])
wildfire["Year"] = wildfire["Incident Start Date"].dt.year

# Clean County column
wildfire["County"] = wildfire["County"].astype(str).str.strip().str.title()

wildfire.head()

/var/folders/sg/bpgt4bnn1qbfgvlc96v1x2m00000gn/T/ipykernel_8050/1728836881.py:6: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,OBJECTID,Incident Name,Incident Start Date,County,Year
0,1,Quail,2020-06-06,Solano,2020
1,2,Quail,2020-06-06,Solano,2020
2,3,Quail,2020-06-06,Solano,2020
3,4,Quail,2020-06-06,Solano,2020
4,5,Quail,2020-06-06,Solano,2020


In [10]:
# Aggregate County and Year to prevent duplication
wildfire = (
    wildfire
    .groupby(["Year", "County"])
    .agg(
        wildfire_records=("OBJECTID", "count"),
        wildfire_incidents=("Incident Name", "nunique")
    )
    .reset_index()
)

wildfire.head()

,Year,County,wildfire_records,wildfire_incidents
0,2013,Riverside,58,1
1,2013,Shasta,222,1
2,2014,Mendocino,11,1
3,2014,San Diego,49,1
4,2014,Siskiyou,250,1


**Merge the two Datasets**

In [11]:
merged = asthma_county.merge(
    wildfire,
    how="left",
    on=["Year", "County"]
)

# If there are no wildfire incidents, replace the value by 0
merged["wildfire_records"] = merged["wildfire_records"].fillna(0).astype(int)
merged["wildfire_incidents"] = merged["wildfire_incidents"].fillna(0).astype(int)
merged.head(10)


,Year,County,Age_Group,total_ed_visits,wildfire_records,wildfire_incidents
0,2013,Alameda,Adult,6653,0,0
1,2013,Alameda,Child,3213,0,0
2,2013,Amador,Adult,195,0,0
3,2013,Amador,Child,12,0,0
4,2013,Butte,Adult,648,0,0
5,2013,Butte,Child,258,0,0
6,2013,Calaveras,Adult,90,0,0
7,2013,Calaveras,Child,27,0,0
8,2013,Colusa,Adult,22,0,0
9,2013,Colusa,Child,32,0,0


The exposure to wildfire level will be categoirzed into four categories:
1. No wildfire: the area does not have wildfire in that particular year
2. Low wildfire: the area has 1 - 19 cases of wildfire in that particular year
3. Moderate wildfire: the area has 20 - 99 cases of wildfire in that particular year
4. High wildfire: the area has more than 100 cases of wildfire in that particular year, at a very high level

In [12]:
# Categorize exposures according to the occurrence
def exposure_category(n):
    if n == 0:
        return "No wildfire"
    elif n < 20:
        return "Low wildfire (1–19)"
    elif n < 100:
        return "Moderate wildfire (20–99)"
    else:
        return "High wildfire (100+)"
        
merged["exposure_cat"] = merged["wildfire_records"].apply(exposure_category)
merged["exposure_cat"].value_counts()


exposure_cat
No wildfire                  745
High wildfire (100+)         130
Low wildfire (1–19)          114
Moderate wildfire (20–99)     61
Name: count, dtype: int64

**Summary of county-level age-adjusted rate and number of ED Visits**

In [13]:
overall_summary = (
    merged
    .groupby("exposure_cat")
    .agg(
        counties = ("County", "nunique"),
        mean_total_ed = ("total_ed_visits", "mean"),    
        median_total_ed = ("total_ed_visits", "median"), 
    )
    .reset_index()
    .sort_values("mean_total_ed", ascending=True)  
)

In [14]:
plot_df1 = merged[merged["exposure_cat"] != "No wildfire"]

fig = px.scatter(
    plot_df1,
    x="wildfire_records",
    y="total_ed_visits",
    color="exposure_cat",
    hover_data=["County", "Year", "total_ed_visits"],
    title="County-Level Asthma ED Rate vs Wildfire Activity (Excluding No Wildfire Counties)",
    labels={
        "wildfire_records": "Wildfire structure records",
        "total_ed_visits": "Total asthma ED visits"
    }
)
fig.show()


In [15]:
trend = (
    merged
    .groupby(["Year", "exposure_cat"])["total_ed_visits"].mean()                              
    .reset_index(name="mean_rate")   # Column is called "mean_rate"
)

trend = trend[trend["exposure_cat"] != "No wildfire"]

fig = px.line(
    trend,
    x="Year",
    y="mean_rate",  # ✓ FIXED - use the actual column name
    color="exposure_cat",
    markers=True,
    title="Asthma ED Visits Over Time by Wildfire Exposure (County-level)",
    labels={"mean_rate": "Mean total ED visits"}  # Add label for clarity
)
fig.show()

## Child vs. Adults

Let's explore whether children are more prone to the impact of wildfire than adult.

In [16]:
# Keep only Child and Adult records
age_filtered = merged[merged["Age_Group"].isin(["Child", "Adult"])].copy()


In [17]:
plot_df = age_filtered[age_filtered["exposure_cat"] != "No wildfire"]
fig = px.scatter(
    plot_df,
    x="wildfire_records",
    y="total_ed_visits",
    color="exposure_cat",
    facet_col="Age_Group",
    hover_data=["County", "Year", "total_ed_visits"],
    title="Asthma ED Rate vs Wildfire Activity, by Age Group (County × Year)",
    labels={
        "wildfire_records": "Wildfire structure records (CAL FIRE, County × Year)",
        "total_ed_visits": "Total asthma ED visits"
    }
)

fig.show()

In [18]:
trend_by_age = (
    age_filtered
    .groupby(["Year", "Age_Group", "exposure_cat"])
    .agg(mean_rate=("total_ed_visits", "mean"))  # Column is called "mean_rate"
    .reset_index()
)
trend_df = trend_by_age[trend_by_age["exposure_cat"] != "No wildfire"]

fig = px.line(
    trend_df,
    x="Year",
    y="mean_rate",  # ✓ FIXED - use the actual column name
    color="exposure_cat",
    facet_row="Age_Group",
    markers=True,
    title="Asthma ED Visits Over Time by Wildfire Exposure and Age Group",
    labels={"mean_rate": "Mean total ED visits"}  # ✓ FIXED label too
)
fig.show()

Suprisingly, my data revealed that counties with no wildfire activity actually have the highest mean asthma ED rates of 52.7 per 10,000 population. 

This counterintuitive finding may be attributed to several factors:
1. Urban vs. Rural: counties without wildfires are likely urban areas, where the asthma rates may be due to housing density, traffic pollution and industrial emissions. On the other hand, wildfire prone counties tend to be more rural.

2. Health Access: populations in wildfire prone counties may have better access to preventive asthma care, such as better air filtration systems installed.

3. Lag in data: my study investigates same year correlations. However in reality, wildfire smoke related asthma conditions might occur after months, or even years after fire events.

We can observe that children consistently show higher vulnerability across all exposure levels. They have approximately 2x higher age-adjusted asthma ED rates than adults. In 2016, we can see that severe fire seasons affect both groups more dramatically than typical years.